In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
# Ensure CUDA is properly used
torch.cuda.empty_cache()

model_path = "./model/Qwen2.5-0.5B-inst/"

# Load model in 4-bit with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_path,
    max_seq_length=2048,  # Adjust based on your dataset needs
    dtype=None,
    load_in_4bit=True
)


==((====))==  Unsloth 2025.3.17: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [15]:

path = "./data.jsonl"
# Load dataset
train_dataset = load_dataset("json", data_files={"train": path})["train"]

def format_chatml(example):
    return {
        "text": (
            "<|im_start|>user " + example["input"] + "<|im_end|>"
            "<|im_start|>assistant " + example["output"] + "<|im_end|>"
        )
    }


train_dataset = train_dataset.map(format_chatml)

def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     truncation=True,
                    padding="max_length",
                    max_length=2048 )

tokenized_datasets = train_dataset.map(tokenize_function, remove_columns=["text", "input", "output"])




Map: 100%|██████████| 67/67 [00:00<00:00, 491.03 examples/s]


In [16]:
train_dataset = tokenized_datasets.with_format("torch")

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


In [18]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Laptop GPU. Max memory = 7.996 GB.
1.273 GB of memory reserved.


In [19]:


trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 67 | Num Epochs = 8 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 737,280/5,000,000,000 (0.01% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.276600
2,5.189800
3,5.358700
4,4.699100
5,4.833300
6,4.952600
7,4.309800
8,4.274900
9,4.510400
10,4.249900


In [20]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

220.799 seconds used for training.
3.68 minutes used for training.
Peak reserved memory = 1.273 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 15.92 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt = "who are you?"



from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(fo, streamer = text_streamer, max_new_tokens = 128)